In [1]:
!python --version

Python 3.8.3


## import statements

In [12]:
import os
import wave
import contextlib
import random
import chardet
import shutil
import re
from pydub import AudioSegment, effects
from pydub.playback import play
from matplotlib import pyplot as plt
import pandas as pd


In [5]:
plt.style.use('fivethirtyeight')
plt.tight_layout()

<Figure size 432x288 with 0 Axes>

## Utility: Detecting Bangla Text
This function will detect whether the given text contains Bangla text or any other text.

In [4]:
# text = 'আমার সোনার - বাংলা ১২৩ ।\nabc123'

def is_bangla(text):
    for c in text:
        val = ord(c)
        if (val<2432 or val>2543) and c not in ' ,।;?!%/(){}[]+-"\'\t\n\r\u200c\u200d':
            return False
    return True

## Utility: Detecting NSWs
This function will detect whether the given text contain Non Standard Words (NSWs), such as numbers and short forms (ডঃ মোঃ etc.)

In [5]:
def has_nsw(text):
    for c in text:
        val = ord(c)
        
        #     bishorgo          digits
        if val == 2435 or 2534 <= val <= 2543: 
                return True
    return False

## Utility: Getting Sentences from Paragraph

In [3]:
def get_sentences(t):
    #import re

    lines = re.split('(\?|।|!)', t.strip())

    n = len(lines)
    if (n & 1): n -= 1
    #print(f'n: {n}')
    
    i = 0
    result = []
    while i < n:
        assert(lines[i+1] in '?।!')
        #print(lines[i] + lines[i+1])
        result.append(lines[i].strip() + lines[i+1])
        i += 2
        
    return result

In [18]:
t = 'যে দেশে শপিং মল, গণপরিবহন, দূর পাল্লার পরিবহন, অফিস-আদালত, বাজার-হাট সবকিছু খুলে দেয়া হয়েছে সেখানে আসলে শিক্ষা প্রতিষ্ঠান বন্ধ করে কিভাবে করোনাভাইরাস বিস্তার নিয়ন্ত্রণ সম্ভব? দ্বিতীয় যে প্রশ্নটি সুরাহা করা কঠিন তা হল একজন শিক্ষার্থী কিভাবে শিক্ষা প্রতিষ্ঠান বন্ধ থাকলে শুধু ঘরে অবস্থান করে করোনাভাইরাস থেকে নিরাপদ থাকবে যেখানে তার পিতা কিংবা মাতা অথবা পিতা-মাতা উভয়কেই জীবিকার তাগিদে প্রতিদিন বাইরে যেতে হচ্ছে। তারা হয়তো গণপরিবহনে গাদাগাদি করে কর্মস্থলে যাচ্ছেন এবং কর্মস্থলে কাজ শেষে আবার বাসায় ফিরে আসছেন! একই ঘরে অবস্থান করা শিক্ষার্থী তার পিতা-মাতার মাধ্যমে করোনাভাইরাসে আক্রান্ত হচ্ছে না তার গ্যারান্টি কে দেবে?'

get_sentences(t)

['যে দেশে শপিং মল, গণপরিবহন, দূর পাল্লার পরিবহন, অফিস-আদালত, বাজার-হাট সবকিছু খুলে দেয়া হয়েছে সেখানে আসলে শিক্ষা প্রতিষ্ঠান বন্ধ করে কিভাবে করোনাভাইরাস বিস্তার নিয়ন্ত্রণ সম্ভব?',
 'দ্বিতীয় যে প্রশ্নটি সুরাহা করা কঠিন তা হল একজন শিক্ষার্থী কিভাবে শিক্ষা প্রতিষ্ঠান বন্ধ থাকলে শুধু ঘরে অবস্থান করে করোনাভাইরাস থেকে নিরাপদ থাকবে যেখানে তার পিতা কিংবা মাতা অথবা পিতা-মাতা উভয়কেই জীবিকার তাগিদে প্রতিদিন বাইরে যেতে হচ্ছে।',
 'তারা হয়তো গণপরিবহনে গাদাগাদি করে কর্মস্থলে যাচ্ছেন এবং কর্মস্থলে কাজ শেষে আবার বাসায় ফিরে আসছেন!',
 'একই ঘরে অবস্থান করা শিক্ষার্থী তার পিতা-মাতার মাধ্যমে করোনাভাইরাসে আক্রান্ত হচ্ছে না তার গ্যারান্টি কে দেবে?']

## Utility: Removing ZWJ and ZWNJ from Bangla Text

In [18]:
"""Utility for context-aware removal of ZWNJ and ZWJ in Bangla text."""

from __future__ import unicode_literals

import io
import re
import unittest


STANDARDIZE_ZW = re.compile(r'(?<=\u09b0)[\u200c\u200d]+(?=\u09cd\u09af)')

DELETE_ZW = re.compile(r'(?<!\u09b0)[\u200c\u200d](?!\u09cd\u09af)')


def RemoveOptionalZW(text):
    """Removes all optional occurrences of ZWNJ or ZWJ from Bangla text.

    The non-printing characters U+200C (ZWNJ) and U+200D (ZWJ) are used in Bangla
    to optionally control the appearance of ligatures, except in one special
    situation: after RA and before VIRAMA+YA, the presence or absence of ZWJ
    (formerly ZWNJ) changes the visual appearance of the involved consonants in a
    meaningful way. This occurrences of ZWJ must be preserved, while all other
    occurrences are advisory and can be removed for most purposes.

    After RA and before VIRAMA+YA, this function changes ZWNJ to ZWJ and preserves
    ZWJ; and removes ZWNJ and ZWJ everywhere else.

    Args:
    text: The text from which the zero-width format controls are to be removed.

    Returns:
    The text with all non-obligatory occurrences of ZWNJ and ZWJ removed.

    """
    text = STANDARDIZE_ZW.sub('\u200D', text)
    text = DELETE_ZW.sub('', text)
    return text


class TestBanglaZeroWidth(unittest.TestCase):

    def test_RemoveOptionalZW(self):
        rally = '\u09b0\u200d\u09cd\u09af\u09be\u09b2\u09c0'
        self.assertEqual(rally, 'র‍্যালী')
        self.assertEqual(RemoveOptionalZW(rally), rally)
        for i in range(len(rally) + 1):
            for zw in ('\u200c', '\u200d', '\u200c\u200d', '\u200d\u200c'):
                text = rally[:i] + zw + rally[i:]
                self.assertEqual(RemoveOptionalZW(text), rally)


if __name__ == '__main__':
    stdin = io.open(0, mode='rt', encoding='utf-8', closefd=False)
    stdout = io.open(1, mode='wt', encoding='utf-8', closefd=False)
    for line in stdin:
        stdout.write(RemoveOptionalZW(line))

# Miscellaneous

## Stripping spaces from filename

In [4]:
path = '/media/arif/Resource/My PC/Tutorial/PhD (Offline)/Resources/Sound Corpus/Bangla Speech Database 2018/Female/Script 07'

In [7]:
files = os.listdir(path)
len(files)

800

In [6]:
for filename in files:
    name, ext = os.path.splitext(filename)
    newname = name.strip() + '.wav'
    os.rename(os.path.join(path, filename), os.path.join(path, newname))

## Copying .wav files from one folder to another

In [13]:
# test
filenames = ['01001', '01002', '01003']
src_dir = '/media/arif/Resource/Etc/Ossian/corpus/bn/speakers/adrita/wav/'
dest_dir = '/home/arif/Desktop/test-folder'
for filename in filenames:
    filename = src_dir + filename + '.wav'
    shutil.copy(filename, dest_dir)
    

In [3]:
filenames = []
with open('google_plus_sust_script.tsv') as f:
    for line in f:
        #print(line)
        items = line.split('\t')
        if len(items) != 3:
            print(line)
        else:
            idx = items[1]
            filenames.append('0' + idx)
        
#filenames = sorted(filenames)
print('total:', len(set(filenames)))
#filenames

src_dir = '/media/arif/Resource/Etc/Ossian/corpus/bn/speakers/kanak/wav/'
dest_dir = '/home/arif/Desktop/kanak-100-original'

for filename in filenames:
    filename = src_dir + filename + '.wav'
    shutil.copy(filename, dest_dir)
    
wav_files = os.listdir(dest_dir)
print(len(wav_files))
for wavfile in wav_files:
    wavfile = wavfile[:-4]
    #print(wavfile)
    filenames.remove(wavfile)
        
print(filenames)























total: 100


FileNotFoundError: [Errno 2] No such file or directory: '/media/arif/Resource/Etc/Ossian/corpus/bn/speakers/kanak/wav/02370.wav'

## Silence Remover

In [7]:
#from pydub import AudioSegment

def detect_leading_silence(sound, silence_threshold=-50.0, chunk_size=10):
    '''
    sound is a pydub.AudioSegment
    silence_threshold in dB
    chunk_size in ms

    iterate over chunks until you find the first one with sound
    '''
    trim_ms = 0 # ms

    assert (chunk_size > 0) # to avoid infinite loop
    while sound[trim_ms:trim_ms+chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
        trim_ms += chunk_size

    if trim_ms > chunk_size: trim_ms -= chunk_size
    if trim_ms > chunk_size: trim_ms -= chunk_size
    return trim_ms

In [ ]:
# single file
filename = '/media/arif/Resource/Etc/Notebooks/02019.wav'
outfile = '/media/arif/Resource/Etc/Notebooks/02019_2.wav'
sound = AudioSegment.from_file(filename, format="wav")

start_trim = detect_leading_silence(sound)
end_trim = detect_leading_silence(sound.reverse())

duration = len(sound)    
trimmed_sound = sound[start_trim:duration-end_trim]

trimmed_sound.export(outfile, format="wav")

In [8]:
# files in a folder
src_path = '/mnt/resource/Tutorial/PhD (Offline)/Resources/Sound Corpus/BracU Speech Data/wavs'
dst_path = '/mnt/resource/Tutorial/PhD (Offline)/Resources/Sound Corpus/BracU Speech Data/wavs_clipped'

files = os.listdir(src_path)
#len(files)
#print(files[0])

i = 0
for filename in files:
    if filename.startswith("."): continue
    src_file = os.path.join(src_path, filename)
    dst_file = os.path.join(dst_path, filename)
    #print(dst_file)
    sound = AudioSegment.from_file(src_file, format='wav')
    
    start_trim = detect_leading_silence(sound)
    end_trim = detect_leading_silence(sound.reverse())
    duration = len(sound)    
    trimmed_sound = sound[start_trim:duration-end_trim]
    trimmed_sound.export(dst_file, format="wav")
    
    #i += 1
    #if i == 5: break

### by Abullah al Mahfuz

In [4]:
import re
from operator import itemgetter
from collections import OrderedDict

file = "tts_script_01.txt"
with open(file, 'r') as fp:
    sound = fp.read()
    pattern = re.compile(
        r'[\u0985-\u098B\u098F-\u0990\u0993-\u0994]|[\u0995-\u09A8\u09AA-\u09B0\u09B2\u09B6-\u09B9\u09CE\u09DC\u09DD\u09DF][\u09BE-\u09C3\u09C7\u09C8\u09CB\u09CC]')
    match = pattern.findall(sound)
    for item in sorted(match):
        print(item)
    t = 0
    dic2 = {}
    while t < len(match):
        ll = match.count(match[t])
        dic2[match[t]] = ll
        t += 1

    sorted_keys = OrderedDict(sorted(dic2.items(), key=itemgetter(0)))

    for key, value in sorted_keys.items():
        print(key, "=", value)


অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
অ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ
আ


ছা
ছা
ছা
ছা
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছি
ছু
ছু
ছু
ছু
ছু
ছু
ছু
ছু
ছু
ছু
ছু
ছু
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
ছে
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জা
জ

নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নী
নু
নু
নু
নু
নু
নু
নু
নু
নু
নু
নু
নু
নু
নু
নু
নু
নু
নু
নু
নূ
নূ
নূ
নূ
নূ
নূ
নূ
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নে
নৈ
নৈ
নো
নো
নো
নো
নো
নো
নো
নো
নো
নো
নো
নো
নো
নো
নো
নো
নো
নো
নো
নো
নো
নো
নো
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
পা
প

লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লে
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
লো
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শা
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শি
শী
শী
শী
শী
শী
শী
শী
শী
শী
শী
শী
শু
শু
শু
শু
শু
শু
শু
শু
শু
শু
শু
শু
শু
শু
শু
শু
শু
শু
শু
শু
শু
শু
শূ
শূ
শূ
শূ
শূ
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শে
শ

## Plotting Data Distribution

### Word Frequency Distribution of Text Data

In [14]:
word_frequency = [0] * 30
word_frequency

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [29]:
with open('tts_script_17.txt') as f:
    for line in f:
        n_words = len(line.split())
        #print(n_words)
        word_frequency[n_words] += 1

16
16
10
9
20
5
11
18
12
10
14
8
9
12
9
10
6
17
7
12
17
12
13
7
17
9
9
7
10
8
8
9
11
10
13
6
6
6
20
8
5
16
9
8
7
10
18
11
14
13
7
15
10
6
10
19
11
9
8
7
20
9
18
6
9
10
8
11
9
8
9
9
10
11
17
9
16
5
18
6
7
15
19
16
10
18
15
6
9
10
7
6
6
19
12
17
9
10
9
5
6
9
17
9
16
5
15
12
12
9
15
19
14
6
19
16
13
7
6
13
5
9
14
14
7
6
15
10
9
15
21
7
14
7
8
7
11
7
14
12
14
6
10
8
9
12
9
20
6
17
12
11
19
9
12
15
8
6
8
8
19
15
9
9
16
6
7
10
6
13
11
8
21
10
15
10
15
10
13
15
7
11
13
10
8
21
5
17
8
11
18
12
9
12
7
5
6
10
5
17
11
11
15
6
8
17
11
6
19
13
11
13
15
7
8
17
15
5
6
9
5
11
21
5
11
11
19
21
15
17
16
7
11
15
9
8
11
15
5
9
10
9
16
16
14
15
14
11
16
10
12
5
11
6
7
7
14
9
21
6
7
7
19
11
6
8
5
10
19
21
5
20
13
10
9
9
17
10
8
10
17
7
13
14
13
5
11
6
20
10
15
20
12
11
18
12
10
10
14
8
8
18
7
9
15
13
5
11
6
13
10
8
14
11
21
8
17
14
9
10
13
6
11
7
8
19
10
6
7
8
14
16
8
10
20
8
9
7
10
10
7
10
8
11
9
8
9
8
9
9
7
15
18
15
13
10
9
16
14
15
12
9
10
16
13
10
12
14
12
8
10
13
13
13
9
8
15
14
9
9
9
7
16
14
13
19
7
5

In [30]:
word_frequency

[0,
 0,
 0,
 1,
 516,
 755,
 948,
 1093,
 1121,
 1141,
 1062,
 960,
 887,
 739,
 669,
 566,
 505,
 452,
 343,
 309,
 254,
 192,
 6,
 5,
 9,
 0,
 4,
 0,
 0,
 0]

In [55]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [56]:
#plt.style.use('classic')
plt.figure()
plt.plot(word_frequency)
#plt.hist()
#plt.show()

## Adjusting Amplitude

It is usually desired that all the audio clips of a corpus should be in the same/maximum level of amplitude/loudness. The following code does this. 

Necessary imports:
```
from pydub import AudioSegment, effects
from pydub.playback import play
```

The `apply_gain` lets you set the amplitude of a clip to a desired value (in dBFS). An average amplitude of -20 dBFS could be chosen. 

The `effects.normalize` function can be used to normalize an audio clip (set the amplitude to the highest possible value). 

Use any of the above methods. Code taken from an [issue at pydub repo](https://github.com/jiaaro/pydub/issues/90).

In [14]:
wavdir = '/mnt/resource/Tutorial/PhD (Offline)/Resources/Sound Corpus/BracU Speech Data/wavs_clipped_mono_22.05k'
dest_dir = '/mnt/resource/Tutorial/PhD (Offline)/Resources/Sound Corpus/BracU Speech Data/wavs_clipped_mono_normalized_22.05k'

files = os.listdir(wavdir)

total = 0
for file in files:
    total += 1
    #if total < 1100: continue
    #if total == 3: break
        
    filename = os.path.join(wavdir,file)
    
    clip = AudioSegment.from_file(filename)
    
    #print(clip.dBFS)
    #play(clip)
    
    #clip = effects.normalize(clip) # or
    clip = clip.apply_gain(-20 - clip.dBFS)
    
    #print(clip.dBFS)
    #play(clip)
    
    clip.export(os.path.join(dest_dir, file), format="wav")
    
    
    
print(f'{total} files updated.')

10894 files updated.


## Stereo to Mono

In [10]:
wavdir = '/mnt/resource/Tutorial/PhD (Offline)/Resources/Sound Corpus/BracU Speech Data/wavs_clipped/'
dest_dir = '/mnt/resource/Tutorial/PhD (Offline)/Resources/Sound Corpus/BracU Speech Data/wavs_clipped_mono/'

files = os.listdir(wavdir)

total = 0
for file in files:
    filename = os.path.join(wavdir,file)
    
    clip = AudioSegment.from_file(filename)
    
    if clip.channels == 2:
        clip.set_channels(1)
        total += 1
    
    clip.export(os.path.join(dest_dir, file), format="wav")
    
    
    
print(f'{total} files converted from setereo to mono.')

184 files converted from setereo to mono.


## Removing Long Audio from BracU Corpus

In [9]:
# detect sentences longer than 17 seconds

wavdir = '/mnt/resource/Tutorial/PhD (Offline)/Resources/Sound Corpus/BracU Speech Data/wavs_clipped_mono_normalized_22.05k'
#dest_dir = '/mnt/resource/Tutorial/PhD (Offline)/Resources/Sound Corpus/BracU Speech Data/wavs_clipped_mono/'

files = os.listdir(wavdir)

total = 0
for file in files:
    filename = os.path.join(wavdir,file)
    
    clip = AudioSegment.from_file(filename)
    
    if clip.duration_seconds > 17:
        print(f'{file} -> {clip.duration_seconds:.2f} seconds')
        total += 1

10146.wav -> 23.56 seconds
10170.wav -> 19.19 seconds
10283.wav -> 17.86 seconds
10305.wav -> 40.10 seconds
10306.wav -> 30.29 seconds
10335.wav -> 21.80 seconds
10386.wav -> 31.11 seconds
10406.wav -> 20.26 seconds
10408.wav -> 24.06 seconds
10606.wav -> 19.02 seconds
10683.wav -> 17.22 seconds
10760.wav -> 20.11 seconds
11051.wav -> 21.15 seconds
11583.wav -> 31.58 seconds
11585.wav -> 24.78 seconds
11588.wav -> 31.04 seconds
11590.wav -> 17.85 seconds
11591.wav -> 25.60 seconds
11593.wav -> 22.48 seconds
11595.wav -> 19.54 seconds
11596.wav -> 18.98 seconds
11616.wav -> 23.28 seconds
11625.wav -> 27.09 seconds
11626.wav -> 37.63 seconds
11627.wav -> 29.98 seconds
11628.wav -> 20.50 seconds
11630.wav -> 29.33 seconds
11631.wav -> 20.80 seconds
11635.wav -> 42.81 seconds
11637.wav -> 31.85 seconds
11639.wav -> 18.98 seconds
11641.wav -> 27.05 seconds
11643.wav -> 24.40 seconds
11647.wav -> 21.57 seconds
11649.wav -> 19.32 seconds
11651.wav -> 32.35 seconds
11652.wav -> 25.02 seconds
1

## PESQ Calculation

In [1]:
!pip install pesq

  Created wheel for pesq: filename=pesq-0.0.2-cp38-cp38-linux_x86_64.whl size=205482 sha256=976e0c27f5e854f995637661680b2463697515f83bfb0b7021c58f94c1aaf417
  Stored in directory: /home/arif/.cache/pip/wheels/1f/db/0a/a42d098aad156dd9690e7c807280add599f1ac675d08dd5272
Successfully built pesq


In [13]:
from scipy.io import wavfile
from pesq import pesq

wavdir = '/mnt/resource/Etc/ESPNet Samples'

rate, ref = wavfile.read(str(os.path.join(wavdir, 'groundtruth_16k.wav')))
rate, deg = wavfile.read(str(os.path.join(wavdir, 'fs2_pwg_16k.wav')))

print(pesq(rate, ref, deg, 'wb'))
print(pesq(rate, ref, deg, 'nb'))

1.064129114151001
1.1368407011032104


# REVE TTS Script

In [19]:
text = ""
with open('/home/arif/Dropbox/PhD/Resource/Corpus/REVE/reve-tts.txt') as f:
    text = f.read()

text = RemoveOptionalZW(text)
len(text)

70279

In [20]:
lines = get_sentences(text)
len(lines)

1635

In [21]:
print(lines[:5])

['খুব সকাল সকাল ঘুম ভেঙ্গে গেল।', 'ফ্রেশ হয়ে এক কাপ চা নিয়ে ছাদে গিয়ে বসলাম।', 'বহুদিন পর আজ ভোরের সূর্য দেখে মনটা শান্ত হয়ে গেল।', 'কী মোহনিয়া রূপ তার!', 'তার এই নিষ্পাপ সৌন্দর্যের কাছে যেন পৃথিবীর অন্য সব সৌন্দর্য হার মেনে যায়!']


In [15]:
lines_df = pd.DataFrame(lines, index=[i+1 for i in range(len(lines))], columns=['sentences'])
#print(lines_df.head(10))
lines_df.head(10)

,sentences
1,খুব সকাল সকাল ঘুম ভেঙ্গে গেল।
2,ফ্রেশ হয়ে এক কাপ চা নিয়ে ছাদে গিয়ে বসলাম।
3,বহুদিন পর আজ ভোরের সূর্য দেখে মনটা শান্ত হয়ে গেল।
4,কী মোহনিয়া রূপ তার!
5,তার এই নিষ্পাপ সৌন্দর্যের কাছে যেন পৃথিবীর অন্...
6,অপরূপ লালচে কমলা রঙের আভা ছড়িয়ে সগৌরবে তার আবি...
7,"ঘোর লাগা চোখে তাকিয়ে ছিলাম, চা ঠান্ডা হয়ে যাচ্..."
8,সম্বিত ফিরল বড় ভাইয়ার ডাকে।
9,"""কী রে, তুই!"
10,আজ সূর্য কোন দিক থেকে উঠল?


In [31]:
char_freq = {}
for c in text:
    if c in '\n "\'()*,-.:;!?।': continue
    char_freq[c] = char_freq.get(c, 0) + 1


In [37]:
char_freq_df = pd.DataFrame(char_freq.items(), columns=['character', 'frequency'])
char_freq_df.sort_values('frequency', inplace=True, ascending=False)
char_freq_df.head(50)

,character,frequency
5,া,6235
10,ে,5516
15,র,4151
22,ন,2999
12,্,2747
4,ক,2669
23,ি,2664
2,ব,2256
32,ত,2105
8,ম,1721
